In [1]:
import socket
import datetime
import json
import re
import time
import zlib


# Steps:
# To use command prompts, start the MacOS Terminal app and enter one of the following commands: 
# 1. “ipconfig getifaddr en1” - The system will return the IP address for a wired Ethernet connection. 
# 2. “ipconfig getifaddr en0” - This will return the IP address of your wireless connection.



class Server:
    # address = '192.168.137.39'
    # the ip is your device's ip 
    def __init__(self, windowDict = {"192.168.137.30": ['0', '0']}, ip='10.12.2.58', doorDict={'192.168.137.219': ['0', '0']}, port = 8000):
        #self.password = password
        #self.address = address
        self.ip = ip
        self.port = port
        self.windowDict = windowDict
        self.doorDict = doorDict
        self.people = 0
        self.tempTime = -1
        self.timeOnDoor2 = -1
        self.timeOnDoor1 = -1
        
    
    def start(self):
        
        server = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        server.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
        server.bind((self.ip, self.port))
        server.listen(5) 
        
        print("Started server")
        
        while True:# conn
            conn,addr = server.accept() 
            print(type(conn))
            print(conn)
            
            try:
                data = str(conn.recv(1024).decode())  
                if data:
                    print('Time:',datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                    print('receive:',data)
                    print()
    
                    if self.windowConnection(addr):
                        self.readWindow(addr, data)
                    elif self.doorConnection(addr):
                        self.readDoor(addr, data)
                    else:
                        self.checkCommands(data, conn)
            except ConnectionResetError as e:
                print('closed busy link?')
                break
            conn.close()
    
    # data should be data.decode()
    def checkCommands(self, data, conn):
        if str(self.ip) + ":" + str(self.port) in data:
            self.encode(conn)
        elif data == "GET_ALL_WINDOWS":
            self.getAllWindows(conn)
        elif data == "GET_ALL_DOORS":
            self.getAllDoors(conn)
        elif data == "GET_PEOPLE":
            self.getPeopleCount(conn)
        elif bool(re.search("^ADD_WINDOW", data)):
            self.addWindow(data)
        elif bool(re.search("^ADD_DOOR", data)):
            self.addWindow(data)
     
    def encode(self, conn):
        # Reply as HTTP/1.1 server, saying "HTTP OK" (code 200).
        # sending all this stuff
        conn.send(b"HTTP/1.1 200 OK\n")
        conn.send(b"Access-Control-Allow-Origin: http://localhost:3000\n")
        conn.send(b"Access-Control-Allow-Credentials: true\n")
        conn.send(b"message")
        
        response_headers = {
            'Content-Type': 'text/html; encoding=utf8',
            'Content-Length': 5,
            'Connection': 'close',
        }
    
        response_headers_raw = ''.join('%s: %s\n' % (k, v) for k, v in \
                                                response_headers.items())
        
        
        response_body = [
            '<html><body><h1>Hello, world!</h1>',
            '<p>This page is in location , was requested ' % locals(),
            'using , and with .</p>' % locals(),
            '<p>Request body isr</p>' % locals(),
            '<p>Actual set of headers received:</p>',
            '<ul>',
        ]
        
        response_body.append('</ul></body></html>')
    
        response_body_raw = ''.join(response_body)
        
        conn.send(memoryview(response_headers_raw.encode('utf-8')).tobytes())
        conn.send(b'\n') # to separate headers from body
        conn.send(memoryview(response_body_raw.encode('utf-8')).tobytes())
        
    
    def windowConnection(self, addr):
        if addr[0] in self.windowDict:
            return True
        else:
            return False
        
    def doorConnection(self, addr):
        if addr[0] in self.doorDict:
            return True
        else:
            return False
    
    def getPeopleCount(self, conn):
        data_string = json.dumps(self.people) #data serialized
        #print(self.doorDict)
        #data_loaded = json.loads(data) #data loaded
        conn.send(data_string.encode())
    
    def getAllWindows(self, conn):
        data_string = json.dumps(self.windowDict) #data serialized
        #print(self.doorDict)
        #data_loaded = json.loads(data) #data loaded
        conn.send(data_string.encode())
        
    def getAllDoors(self, conn):
        data_string = json.dumps(self.doorDict) 
        #print(self.doorDict)
        conn.send(data_string.encode())
    
    def addWindow(self, data):
        #print(data)
        newWindowAddr = re.search("(?<=^ADD_WINDOW)[0-9\.]*", data).group(0)
        self.windowDict[newWindowAddr] = 0
        
    def addDoor(self, data):
        newWindowAddr = re.search("(?<=^ADD_DOOR)[0-9\.]*", data).group(0)
        self.doorDict[newWindowAddr] = 0
        
    # data should be data.decode()
    def readDoor(self, addr, data):
        if self.doorDict[addr[0]][1] == "1" and self.doorDict[addr[0]][0] == "1":
            if re.search("(?<=Door_1:)[0-9]*", data) != None:
                self.doorDict[addr[0]][0] = re.search("(?<=Door_1:)[0-9]*", data).group(0)
            else:
                self.doorDict[addr[0]][1] = re.search("(?<=Door_2:)[0-9]*", data).group(0)
            
            if self.doorDict[addr[0]][0] == "0":
                self.people += self.convertDatetime(datetime.datetime.now()) - self.timeOnDoor1;
            else:
                self.people -= self.convertDatetime(datetime.datetime.now()) - self.timeOnDoor2;
        else:
            if re.search("(?<=Door_1:)[0-9]*", data) != None:
                self.doorDict[addr[0]][0] = re.search("(?<=Door_1:)[0-9]*", data).group(0)
                self.timeOnDoor1 = self.convertDatetime(datetime.datetime.now())
                
            if re.search("(?<=Door_2:)[0-9]*", data) != None:
                self.doorDict[addr[0]][1] = re.search("(?<=Door_2:)[0-9]*", data).group(0)
                self.timeOnDoor2 = self.convertDatetime(datetime.datetime.now())
        
    # data should be data.decode()
    def readWindow(self, addr, data):
        if re.search("(?<=Pressure_1:)[0-9]*", data) != None:
            self.windowDict[addr[0]][0] = re.search("(?<=Pressure_1:)[0-9]*", data).group(0)
        if re.search("(?<=Pressure_2:)[0-9]*", data) != None:
            self.windowDict[addr[0]][1] = re.search("(?<=Pressure_2:)[0-9]*", data).group(0)
            
    def convertDatetime(self, dt):
        return time.mktime((dt).timetuple()) + (dt).microsecond/1e6 



In [2]:
import datetime
a = datetime.datetime.now()#.strftime('%Y-%m-%d %H:%M:%S')

In [3]:
b = datetime.datetime.now()


In [ ]:
print(a)
print(b)
print(b-a)

In [2]:
import json
a = "asdf"
json.dumps(a).encode()

ERROR: Invalid requirement: '—'
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import time
time.mktime((b).timetuple()) + (b).microsecond/1e6 - time.mktime((a).timetuple()) + (a).microsecond/1e6

In [6]:
def encode(self, conn):
        # Reply as HTTP/1.1 server, saying "HTTP OK" (code 200).
        # sending all this stuff
        conn.send(b"HTTP/1.1 200 OK\n")
        conn.send(b"Access-Control-Allow-Origin: http://localhost:3000\n")
        conn.send(b"Access-Control-Allow-Credentials: true\n")
        conn.send(b"message")
        
        response_headers = {
            'Content-Type': 'text/html; encoding=utf8',
            'Content-Length': 5,
            'Connection': 'close',
        }
    
        response_headers_raw = ''.join('%s: %s\n' % (k, v) for k, v in \
                                                response_headers.items())
        
        
        response_body = [
            '<html><body><h1>Hello, world!</h1>',
            '<p>This page is in location , was requested ' % locals(),
            'using , and with .</p>' % locals(),
            '<p>Request body isr</p>' % locals(),
            '<p>Actual set of headers received:</p>',
            '<ul>',
        ]
        
        response_body.append('</ul></body></html>')
    
        response_body_raw = ''.join(response_body)
        
        conn.send(memoryview(response_headers_raw.encode('utf-8')).tobytes())
        conn.send(b'\n') # to separate headers from body
        conn.send(memoryview(response_body_raw.encode('utf-8')).tobytes())
        

ERROR: Could not find a version that satisfies the requirement scipy.io (from versions: none)
ERROR: No matching distribution found for scipy.io
Note: you may need to restart the kernel to use updated packages.


In [13]:
a = {}
a["adsf"] = 3
list(a.keys())[0]

'adsf'

In [2]:
int(3.4)

3

In [1]:
import socket
import datetime
import json
import re
import time
import zlib


# Steps:
# To use command prompts, start the MacOS Terminal app and enter one of the following commands: 
# 1. “ipconfig getifaddr en1” - The system will return the IP address for a wired Ethernet connection. 
# 2. “ipconfig getifaddr en0” - This will return the IP address of your wireless connection.



class Server:
    # address = '192.168.137.39'
    # the ip is your device's ip 
    def __init__(self, windowDict = {"192.168.137.179": ['0', '0']}, ip='192.168.137.181', doorDict={'192.168.137.69': ['0', '0']}, port = 8000):
        #self.password = password
        #self.address = address
        self.ip = ip
        self.port = port
        self.windowDict = windowDict
        self.doorDict = doorDict
        self.people = 0
        self.tempTime = -1
        self.timeOnDoor2 = -1
        self.timeOnDoor1 = -1
        
    
    def start(self):
        
        server = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        server.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
        server.bind((self.ip, self.port))
        server.listen(5) 
        
        print("Started server")
        
        while True:# conn
            conn,addr = server.accept() 
        #print(conn)
            
            try:
                data = str(conn.recv(1024).decode())  
                if data:
                    print('Time:',datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                    print('receive:',data)
                    print()
    
                    if self.windowConnection(addr):
                        self.readWindow(addr, data)
                    elif self.doorConnection(addr):
                        self.readDoor(addr, data)
                    else:
                        self.checkCommands(data, conn)
            except ConnectionResetError as e:
                print('closed busy link?')
                break
            conn.close()
    
    # data should be data.decode()
    def checkCommands(self, data, conn):
        if str(self.ip) + ":" + str(self.port) in data:
            #self.encode(conn)
            conn.send("Server says hello!".encode())
        elif data == "GET_ALL_WINDOWS":
            self.getAllWindows(conn)
        elif data == "GET_ALL_DOORS":
            self.getAllDoors(conn)
        elif data == "GET_PEOPLE":
            self.getPeopleCount(conn)
        elif bool(re.search("^ADD_WINDOW", data)):
            self.addWindow(data)
        elif bool(re.search("^ADD_DOOR", data)):
            self.addWindow(data)
     
    
    
    def windowConnection(self, addr):
        if addr[0] in self.windowDict:
            return True
        else:
            return False
        
    def doorConnection(self, addr):
        if addr[0] in self.doorDict:
            return True
        else:
            return False
    
    def getPeopleCount(self, conn):
        data_string = json.dumps(int(self.people)) #data serialized
        #print(self.doorDict)
        #data_loaded = json.loads(data) #data loaded
        conn.send(data_string.encode())
    
    def getAllWindows(self, conn):
        data_string = json.dumps(self.windowDict[list(self.windowDict.keys())[0]]) #data serialized
        #print(self.doorDict)
        #data_loaded = json.loads(data) #data loaded
        conn.send(data_string.encode())
        
    def getAllDoors(self, conn):
        data_string = json.dumps(self.doorDict) 
        #print(self.doorDict)
        conn.send(data_string.encode())
    
    def addWindow(self, data):
        #print(data)
        newWindowAddr = re.search("(?<=^ADD_WINDOW)[0-9\.]*", data).group(0)
        self.windowDict[newWindowAddr] = 0
        
    def addDoor(self, data):
        newWindowAddr = re.search("(?<=^ADD_DOOR)[0-9\.]*", data).group(0)
        self.doorDict[newWindowAddr] = 0
        
    # data should be data.decode()
    def readDoor(self, addr, data):
        if self.doorDict[addr[0]][1] == "1" and self.doorDict[addr[0]][0] == "1":
            if re.search("(?<=Door_1:)[0-9]*", data) != None:
                self.doorDict[addr[0]][0] = re.search("(?<=Door_1:)[0-9]*", data).group(0)
            else:
                self.doorDict[addr[0]][1] = re.search("(?<=Door_2:)[0-9]*", data).group(0)
            
            if self.doorDict[addr[0]][0] == "0":
                self.people += self.convertDatetime(datetime.datetime.now()) - self.timeOnDoor1;
            else:
                self.people -= self.convertDatetime(datetime.datetime.now()) - self.timeOnDoor2;
        else:
            if re.search("(?<=Door_1:)[0-9]*", data) != None:
                self.doorDict[addr[0]][0] = re.search("(?<=Door_1:)[0-9]*", data).group(0)
                self.timeOnDoor1 = self.convertDatetime(datetime.datetime.now())
                
            if re.search("(?<=Door_2:)[0-9]*", data) != None:
                self.doorDict[addr[0]][1] = re.search("(?<=Door_2:)[0-9]*", data).group(0)
                self.timeOnDoor2 = self.convertDatetime(datetime.datetime.now())
        if self.people < 0:
            self.people = 0
        
        
    # data should be data.decode()
    def readWindow(self, addr, data):
        if re.search("(?<=Pressure_1:)[0-9]*", data) != None:
            self.windowDict[addr[0]][0] = re.search("(?<=Pressure_1:)[0-9]*", data).group(0)
        if re.search("(?<=Pressure_2:)[0-9]*", data) != None:
            self.windowDict[addr[0]][1] = re.search("(?<=Pressure_2:)[0-9]*", data).group(0)
            
    def convertDatetime(self, dt):
        return time.mktime((dt).timetuple()) + (dt).microsecond/1e6 



In [2]:
test = Server()

In [ ]:
test.start()

Started server
Time: 2023-12-03 20:50:20
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:21
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:22
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0


Time: 2023-12-03 20:50:23
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:24
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:25
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:26
receive: Door_2:1

Time: 2023-12-03 20:50:26
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:27
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:28
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:29
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:30
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:50:31
receive: Door_2:0

Time: 2023-12-03 20:50:31
receive: Sensor_Pressure_1:0Sensor_Pressure

Time: 2023-12-03 20:52:20
receive: Door_2:1

Time: 2023-12-03 20:52:20
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:21
receive: Door_1:1

Time: 2023-12-03 20:52:22
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:23
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:26
receive: Door_1:0

Time: 2023-12-03 20:52:26
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:27
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:28
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:29
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:31
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:32
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:33
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:34
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:52:35
receive: Senso

Time: 2023-12-03 20:54:14
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:54:15
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:54:15
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:54:15
receive: GET_PEOPLE

Time: 2023-12-03 20:54:16
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0


Time: 2023-12-03 20:54:17
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:54:17
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:54:17
receive: GET_PEOPLE

Time: 2023-12-03 20:54:18
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:54:19
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:54:19
receive: GET_PEOPLE

Time: 2023-12-03 20:54:19
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:54:20
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:54:21
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:54:21
receive: GET_PEOPLE

Time: 2023-12-03 20:54:21
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Tim

Time: 2023-12-03 20:55:20
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:55:21
receive: Sensor_Pressure_1:2363Sensor_Pressure_2:0

Time: 2023-12-03 20:55:21
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:55:21
receive: GET_PEOPLE

Time: 2023-12-03 20:55:22
receive: Sensor_Pressure_1:4095Sensor_Pressure_2:0

Time: 2023-12-03 20:55:22
receive: Door_2:0


Time: 2023-12-03 20:55:23
receive: Sensor_Pressure_1:4095Sensor_Pressure_2:0

Time: 2023-12-03 20:55:23
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:55:23
receive: GET_PEOPLE

Time: 2023-12-03 20:55:24
receive: Sensor_Pressure_1:4095Sensor_Pressure_2:0

Time: 2023-12-03 20:55:25
receive: Sensor_Pressure_1:4095Sensor_Pressure_2:0

Time: 2023-12-03 20:55:25
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:55:25
receive: GET_PEOPLE

Time: 2023-12-03 20:55:27
receive: Sensor_Pressure_1:4095Sensor_Pressure_2:0

Time: 2023-12-03 20:55:27
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:55:27
receive: GET_PEOPLE

Time: 2023-12-0

Time: 2023-12-03 20:56:30
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:56:31
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:56:32
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:56:32
receive: GET_PEOPLE

Time: 2023-12-03 20:56:32
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:56:33
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:56:34
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:56:34
receive: GET_PEOPLE

Time: 2023-12-03 20:56:34
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:56:35
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:56:36
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:56:36
receive: GET_PEOPLE

Time: 2023-12-03 20:56:36
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:56:37
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:56:38
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:56:38
receive: GET_PEOPLE

Time

Time: 2023-12-03 20:57:37
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:57:38
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:57:38
receive: GET_PEOPLE

Time: 2023-12-03 20:57:38
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:57:40
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:57:40
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:57:40
receive: GET_PEOPLE

Time: 2023-12-03 20:57:40
receive: Door_2:0


Time: 2023-12-03 20:57:41
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:57:41
receive: Door_1:0

Time: 2023-12-03 20:57:42
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:57:42
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:57:42
receive: GET_PEOPLE

Time: 2023-12-03 20:57:43
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:57:44
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:57:44
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:57:44
receive: 

Time: 2023-12-03 20:58:40
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:58:40
receive: GET_PEOPLE

Time: 2023-12-03 20:58:40
receive: Sensor_Pressure_1:4095Sensor_Pressure_2:0

Time: 2023-12-03 20:58:41
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:58:42
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:58:42
receive: GET_PEOPLE

Time: 2023-12-03 20:58:42
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:58:43
receive: Door_2:1

Time: 2023-12-03 20:58:44
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:58:44
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:58:44
receive: GET_PEOPLE

Time: 2023-12-03 20:58:45
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:58:46
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:58:46
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:58:46
receive: GET_PEOPLE

Time: 2023-12-03 20:58:47
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:58:48
recei

Time: 2023-12-03 20:59:43
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:59:44
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:59:44
receive: GET_PEOPLE

Time: 2023-12-03 20:59:44
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:59:45
receive: Door_1:0

Time: 2023-12-03 20:59:45
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:59:46
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:59:46
receive: GET_PEOPLE

Time: 2023-12-03 20:59:46
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:59:47
receive: Door_2:1

Time: 2023-12-03 20:59:47
receive: Door_1:1

Time: 2023-12-03 20:59:47
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:59:48
receive: GET_ALL_WINDOWS

Time: 2023-12-03 20:59:48
receive: GET_PEOPLE

Time: 2023-12-03 20:59:48
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:59:49
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 20:59:50
receive: Door_1:0

Time: 2023-12-03 21:00:47
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 21:00:48
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 21:00:48
receive: GET_ALL_WINDOWS

Time: 2023-12-03 21:00:48
receive: GET_PEOPLE

Time: 2023-12-03 21:00:48
receive: Door_1:0

Time: 2023-12-03 21:00:49
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 21:00:50
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

Time: 2023-12-03 21:00:50
receive: GET_ALL_WINDOWS

Time: 2023-12-03 21:00:50
receive: GET_PEOPLE

Time: 2023-12-03 21:00:51
receive: Sensor_Pressure_1:0Sensor_Pressure_2:0

